In [14]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import librosa, librosa.display
import IPython.display as ipd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, models
from torchvision.datasets import ImageFolder

In [2]:
CREMA="../dataset/AudioWAV"
dir_list = os.listdir(CREMA)
dir_list.sort()
print(dir_list[0:10])

['1001_DFA_ANG_XX.wav', '1001_DFA_DIS_XX.wav', '1001_DFA_FEA_XX.wav', '1001_DFA_HAP_XX.wav', '1001_DFA_NEU_XX.wav', '1001_DFA_SAD_XX.wav', '1001_IEO_ANG_HI.wav', '1001_IEO_ANG_LO.wav', '1001_IEO_ANG_MD.wav', '1001_IEO_DIS_HI.wav']


In [1]:



file_path="../dataset/AudioWAV/1001_DFA_ANG_XX.wav"


audio, sr = librosa.load(file_path)
plt.figure(figsize=(15, 5))


# Lets play the audio 
ipd.Audio(file_path)
print(sr)
generate_log_mel_spectrogram(audio, output_file)

22050


<Figure size 1500x500 with 0 Axes>

In [4]:
gender = []
emotion = []
path = []
female = [1002,1003,1004,1006,1007,1008,1009,1010,1012,1013,1018,1020,1021,1024,1025,1028,1029,1030,1037,1043,1046,1047,1049,
          1052,1053,1054,1055,1056,1058,1060,1061,1063,1072,1073,1074,1075,1076,1078,1079,1082,1084,1089,1091]

for i in dir_list: 
    part = i.split('_')
    if int(part[0]) in female:
        temp = 'female'
    else:
        temp = 'male'
    gender.append(temp)
    if part[2] == 'SAD' and temp == 'male':
        emotion.append('male_sad')
    elif part[2] == 'ANG' and temp == 'male':
        emotion.append('male_angry')
    elif part[2] == 'DIS' and temp == 'male':
        emotion.append('male_disgust')
    elif part[2] == 'FEA' and temp == 'male':
        emotion.append('male_fear')
    elif part[2] == 'HAP' and temp == 'male':
        emotion.append('male_happy')
    elif part[2] == 'NEU' and temp == 'male':
        emotion.append('male_neutral')
    elif part[2] == 'SAD' and temp == 'female':
        emotion.append('female_sad')
    elif part[2] == 'ANG' and temp == 'female':
        emotion.append('female_angry')
    elif part[2] == 'DIS' and temp == 'female':
        emotion.append('female_disgust')
    elif part[2] == 'FEA' and temp == 'female':
        emotion.append('female_fear')
    elif part[2] == 'HAP' and temp == 'female':
        emotion.append('female_happy')
    elif part[2] == 'NEU' and temp == 'female':
        emotion.append('female_neutral')
    else:
        emotion.append('Unknown')
    path.append(CREMA + i)
    
CREMA_df = pd.DataFrame(emotion, columns = ['labels'])
CREMA_df['source'] = 'CREMA'
CREMA_df = pd.concat([CREMA_df,pd.DataFrame(path, columns = ['path'])],axis=1)
CREMA_df.labels.value_counts()

labels
male_angry        587
male_disgust      587
male_fear         587
male_happy        587
male_sad          587
female_angry      548
female_disgust    548
female_fear       548
female_happy      548
female_sad        545
male_neutral      503
female_neutral    465
Name: count, dtype: int64

In [80]:
import os
import shutil

# Define the source directory where your .wav files are located
source_dir = CREMA

# Define the base directory where you want to organize your data
base_dir = "../OrganizedData/"

# Create base directory if it doesn't exist
os.makedirs(base_dir, exist_ok=True)

# Iterate over files in the source directory
for file_name in os.listdir(source_dir):
    if file_name.endswith('.wav'):
        # Parse file name to extract speaker number and emotion
        speaker_number, _, emotion, _ = file_name.split('_')

        # Create directories if they don't exist
        speaker_dir = os.path.join(base_dir, speaker_number)
        os.makedirs(speaker_dir, exist_ok=True)

        emotion_dir = os.path.join(speaker_dir, emotion)
        os.makedirs(emotion_dir, exist_ok=True)

        # Move the file to the corresponding directory
        source_path = os.path.join(source_dir, file_name)
        destination_path = os.path.join(emotion_dir, file_name)
        shutil.copy(source_path, destination_path)

print("Dataset organized successfully!")


Dataset organized successfully!


In [81]:
base_dir = "../OrganizedData/1001"
subfolder_count = sum(os.path.isdir(os.path.join(base_dir, d)) for d in os.listdir(base_dir))

print("Number of subfolders:", subfolder_count)

Number of subfolders: 6


In [5]:
len(female)

43

In [41]:

# Define the directory you want to count files in
folder_path = "../OrganizedData/1001/HAP"

# Count the number of files in the folder
file_count = sum([1 for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])

print("Number of files in the folder:", file_count)

Number of files in the folder: 14


In [46]:
import os
import shutil

# Define the source directory where your organized data is located
source_dir = "../OrganizedData"

# Define the target directories TrainNeutral and TestNeutral
train_neutral_dir = "../CreatedDatasets/TrainSad"
test_neutral_dir = "../CreatedDatasets/TestSad"

# Create TrainNeutral and TestNeutral directories if they don't exist
os.makedirs(train_neutral_dir, exist_ok=True)
os.makedirs(test_neutral_dir, exist_ok=True)

# Create TrainNeutral and TestNeutral directories if they don't exist
os.makedirs(train_neutral_dir, exist_ok=True)
os.makedirs(test_neutral_dir, exist_ok=True)

# Traverse through the directory structure
for root, dirs, files in os.walk(source_dir):
    for file in files:
        if file.endswith('.wav'):
            speaker_id, sentence_id, emotion, _ = file.split('_')
            if emotion == "SAD":  # Check if the emotion is neutral ("NEU")
                if sentence_id in ["TSI", "WSI"]:  # Check if sentence ID is "TSI" or "WSI"
                    destination_dir = test_neutral_dir
                else:
                    destination_dir = train_neutral_dir

                # Create the corresponding directory in the target directory
                relative_path = os.path.relpath(root, source_dir)
                target_dir = os.path.join(destination_dir, relative_path)
                os.makedirs(target_dir, exist_ok=True)

                # Move the file to the target directory
                source_path = os.path.join(root, file)
                target_path = os.path.join(target_dir, file)
                shutil.copy(source_path, target_path)

print("Files copied successfully!")

Files copied successfully!


#generate logmel

In [6]:
import os
import matplotlib.pyplot as plt
import librosa, librosa.display
import IPython.display as ipd
import numpy as np


import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.models as models
from torchvision.transforms import Resize, Normalize, ToTensor

In [51]:
# Assume the spectrogram images are organized by speaker in the input directory

#audio_dir = r"C:\Users\pragy\OneDrive\Desktop\Major\tdnn\Dataset"
audio_dir = r"..\CreatedDatasets\TestSad"

# Specify the output root directory where speaker folders will be created

#output_root_dir = r"C:\Users\pragy\OneDrive\Desktop\Major\tdnn\log spectrogram images"
output_root_dir = r"..\CreatedDatasets\logMelspectrogram_TestSad"
#print("hi1")

In [52]:
# Function to generate and save log mel spectrogram from an audio file
def generate_log_mel_spectrogram(audio_file, output_path):
    # Load the audio file (implementation not shown for brevity)

    # Generate and save the log mel spectrogram
    # (implementation not shown for brevity)

# Ensure the output root directory exists
    y, sr = librosa.load(audio_file, sr=16000)
    
    # Generate log mel spectrogram
    log_mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, center=False)

      # Convert to decibels (log scale)
    log_mel_spectrogram_db = librosa.power_to_db(log_mel_spectrogram, ref=np.max)

      # Create and save the spectrogram image
    plt.figure(figsize=(10, 5))
    #librosa.display.specshow(log_mel_spectrogram_db, sr=sr, x_axis='time', y_axis='mel')
    img=librosa.display.specshow(log_mel_spectrogram_db, sr=sr)
    #plt.colorbar(format='%+2.0f dB')
    #plt.title('Log Mel Spectrogram')
    plt.tight_layout()
    #plt.show()

  # Save the image as a PNG file
    plt.savefig(output_path, bbox_inches='tight')
    #plt.show()
    plt.close()

os.makedirs(output_root_dir, exist_ok=True)

# Loop through all audio files in the directory
for root, _, files in os.walk(audio_dir):
        #print('hi-2')
        for file in files:
            #print('hi-2')
            if file.endswith('.wav'):
                #print('hi-3')
               # Extract the speaker name from the directory structure
                speaker_name = os.path.basename(os.path.dirname(root))                                 
                #print(speaker_name)
            # Create a subdirectory for the speaker in the output root directory
                output_speaker_dir = os.path.join(output_root_dir, speaker_name)
                os.makedirs(output_speaker_dir, exist_ok=True)

                audio_file = os.path.join(root, file)
            
            # Generate the output file path by replacing the extension and prepending the speaker folder
                output_file = os.path.join(output_speaker_dir, file.replace('.wav', '_log_mel.png'))

            # Generate and save the log mel spectrogram
                generate_log_mel_spectrogram(audio_file, output_file)
                print(f"Generated Log Mel spectrogram for {file} and saved to {output_file}")


Generated Log Mel spectrogram for 1001_TSI_SAD_XX.wav and saved to ..\CreatedDatasets\logMelspectrogram_TestSad\1001\1001_TSI_SAD_XX_log_mel.png
Generated Log Mel spectrogram for 1001_WSI_SAD_XX.wav and saved to ..\CreatedDatasets\logMelspectrogram_TestSad\1001\1001_WSI_SAD_XX_log_mel.png
Generated Log Mel spectrogram for 1002_TSI_SAD_XX.wav and saved to ..\CreatedDatasets\logMelspectrogram_TestSad\1002\1002_TSI_SAD_XX_log_mel.png
Generated Log Mel spectrogram for 1002_WSI_SAD_XX.wav and saved to ..\CreatedDatasets\logMelspectrogram_TestSad\1002\1002_WSI_SAD_XX_log_mel.png
Generated Log Mel spectrogram for 1003_TSI_SAD_XX.wav and saved to ..\CreatedDatasets\logMelspectrogram_TestSad\1003\1003_TSI_SAD_XX_log_mel.png
Generated Log Mel spectrogram for 1003_WSI_SAD_XX.wav and saved to ..\CreatedDatasets\logMelspectrogram_TestSad\1003\1003_WSI_SAD_XX_log_mel.png
Generated Log Mel spectrogram for 1004_TSI_SAD_XX.wav and saved to ..\CreatedDatasets\logMelspectrogram_TestSad\1004\1004_TSI_SAD_

Generated Log Mel spectrogram for 1030_WSI_SAD_XX.wav and saved to ..\CreatedDatasets\logMelspectrogram_TestSad\1030\1030_WSI_SAD_XX_log_mel.png
Generated Log Mel spectrogram for 1031_TSI_SAD_XX.wav and saved to ..\CreatedDatasets\logMelspectrogram_TestSad\1031\1031_TSI_SAD_XX_log_mel.png
Generated Log Mel spectrogram for 1031_WSI_SAD_XX.wav and saved to ..\CreatedDatasets\logMelspectrogram_TestSad\1031\1031_WSI_SAD_XX_log_mel.png
Generated Log Mel spectrogram for 1032_TSI_SAD_XX.wav and saved to ..\CreatedDatasets\logMelspectrogram_TestSad\1032\1032_TSI_SAD_XX_log_mel.png
Generated Log Mel spectrogram for 1032_WSI_SAD_XX.wav and saved to ..\CreatedDatasets\logMelspectrogram_TestSad\1032\1032_WSI_SAD_XX_log_mel.png
Generated Log Mel spectrogram for 1033_TSI_SAD_XX.wav and saved to ..\CreatedDatasets\logMelspectrogram_TestSad\1033\1033_TSI_SAD_XX_log_mel.png
Generated Log Mel spectrogram for 1033_WSI_SAD_XX.wav and saved to ..\CreatedDatasets\logMelspectrogram_TestSad\1033\1033_WSI_SAD_

Generated Log Mel spectrogram for 1060_TSI_SAD_XX.wav and saved to ..\CreatedDatasets\logMelspectrogram_TestSad\1060\1060_TSI_SAD_XX_log_mel.png
Generated Log Mel spectrogram for 1060_WSI_SAD_XX.wav and saved to ..\CreatedDatasets\logMelspectrogram_TestSad\1060\1060_WSI_SAD_XX_log_mel.png
Generated Log Mel spectrogram for 1061_TSI_SAD_XX.wav and saved to ..\CreatedDatasets\logMelspectrogram_TestSad\1061\1061_TSI_SAD_XX_log_mel.png
Generated Log Mel spectrogram for 1061_WSI_SAD_XX.wav and saved to ..\CreatedDatasets\logMelspectrogram_TestSad\1061\1061_WSI_SAD_XX_log_mel.png
Generated Log Mel spectrogram for 1062_TSI_SAD_XX.wav and saved to ..\CreatedDatasets\logMelspectrogram_TestSad\1062\1062_TSI_SAD_XX_log_mel.png
Generated Log Mel spectrogram for 1062_WSI_SAD_XX.wav and saved to ..\CreatedDatasets\logMelspectrogram_TestSad\1062\1062_WSI_SAD_XX_log_mel.png
Generated Log Mel spectrogram for 1063_TSI_SAD_XX.wav and saved to ..\CreatedDatasets\logMelspectrogram_TestSad\1063\1063_TSI_SAD_

In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [21]:
#####TRAINING#############
# Define a ResNet model for speaker recognition
class ResNetModel(nn.Module):
    def __init__(self, num_classes):
            super(ResNetModel, self).__init__()
            self.resnet = models.resnet18(weights=True)
    #   Modify the last fully connected layer for your classification task
            in_features = self.resnet.fc.in_features
            self.resnet.fc = nn.Linear(in_features, num_classes)
    def forward(self, x):
            return self.resnet(x)

# Hyperparameters
batch_size = 16
learning_rate = 0.001
num_epochs = 30
num_classes = 91 # Replace with the number of classes in your dataset

# Initialize the ResNet model
model = ResNetModel(num_classes).to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Load the dataset using torchvision's ImageFolder
data_folder = r"..\CreatedDatasets\logMelspectrogram_TrainNeutral"# Update with the path to your spectrogram images
transform = transforms.Compose([
    transforms.Resize((490, 990)),  # Resize images to new_height x new_width
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize images
])
dataset = ImageFolder(data_folder, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Training loop
for epoch in range(num_epochs):
    print("epoch", epoch)
    total_loss = 0.0
    total_correct = 0
    total_samples = 0
    for data, labels in dataloader:
        print("labels",labels)
        optimizer.zero_grad()
        data=data.to(device, dtype=torch.float)
        labels=labels.to(device)
        outputs = model(data)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)

average_loss = total_loss / len(dataloader)
accuracy = total_correct / total_samples
print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {average_loss}, Accuracy: {accuracy * 100:.2f}%")

print("training complete")

epoch 0
labels tensor([58, 73, 80, 62,  3, 74, 76, 54, 19, 79, 56, 28, 16, 25,  8, 63])
labels tensor([68, 73, 40, 14,  8, 63, 58, 24, 33, 39, 58,  4, 28, 16, 71, 53])
labels tensor([43, 37, 69,  2, 55, 34, 70, 32, 53, 64, 22, 67, 72,  6, 27, 12])
labels tensor([35, 78, 39, 31, 46, 43, 25, 69, 21, 68, 78, 40, 69, 18, 46, 10])
labels tensor([80, 59,  6, 53, 17, 18, 66, 18, 27,  4, 31, 78, 20, 51, 13,  1])
labels tensor([42, 36, 65, 22, 49, 74, 59, 37, 28, 49, 66, 53, 62, 60, 64, 45])
labels tensor([40, 36, 36, 59, 58, 30, 25, 21, 35, 35,  9, 60, 41, 17, 38, 39])
labels tensor([44, 79, 76, 49,  1, 68, 16, 47, 50, 17, 46, 48,  7, 54,  3, 37])
labels tensor([58, 32, 51, 15, 23, 54, 72, 19, 27, 49, 58, 24,  2,  4,  5, 35])
labels tensor([45, 51, 30, 68, 36, 20, 38, 65, 80, 57, 41, 21, 77, 69, 30, 61])
labels tensor([27, 69,  5,  2, 24, 30,  8, 41, 71, 52, 80, 26, 60,  0, 25, 39])
labels tensor([23, 71, 72, 80,  9, 11, 79, 15, 13,  6, 40, 59,  9, 67,  3, 72])
labels tensor([47, 21, 46, 24, 4

labels tensor([69, 13, 71, 31, 33, 66, 22,  6,  4, 58,  9, 42, 56, 77, 71,  0])
labels tensor([62, 58, 59, 71, 37, 42, 36, 24, 26, 47, 33, 22,  0,  8, 12, 65])
labels tensor([ 6,  7, 21, 15, 60, 65,  5, 64, 29, 60, 32, 20, 72, 13, 62, 23])
labels tensor([ 2, 74, 73, 37,  7, 77,  9, 79, 62, 59, 60, 71, 29, 34,  4, 50])
labels tensor([53, 75, 78, 50, 77, 39, 48, 62, 17, 64, 51, 13, 17, 17, 63, 34])
labels tensor([31, 17, 54, 68, 54, 73, 48, 46, 44, 18,  6, 38, 11, 80,  4, 67])
labels tensor([22, 60, 61, 67, 79, 69, 30, 28,  5, 71, 80, 80, 23, 73, 21, 15])
labels tensor([52, 72, 50, 78, 72, 72, 56, 38, 80, 10, 70, 44, 57, 55, 65, 17])
labels tensor([41, 55, 78, 75, 79, 55, 30, 26, 11, 45, 75, 11, 68, 22, 21, 42])
labels tensor([57, 57, 13, 19, 50, 50, 47,  4, 78, 20, 61, 37, 32, 35, 54, 13])
labels tensor([71, 77,  0, 53, 44, 28, 46,  2, 42, 35, 28, 18, 66, 76, 17, 61])
labels tensor([65, 12, 15, 49,  3, 46, 45, 31, 47, 46, 40, 15, 25, 12, 49, 10])
labels tensor([71, 57, 38, 60, 67, 74, 4

labels tensor([ 1, 55, 28, 73, 54, 29, 76, 31, 76, 39, 25, 71, 31, 75,  3, 44])
labels tensor([ 5, 75, 24, 47, 65, 71, 79, 36, 45, 25, 62, 15, 55, 45, 26, 65])
labels tensor([69, 40,  7, 47, 23, 30, 20, 65, 50, 49, 11, 18, 75, 68, 62, 26])
labels tensor([ 8,  4,  5, 17, 73, 27, 57, 77, 58, 64, 77, 20,  7, 34, 51, 39])
labels tensor([46, 47, 13, 54, 52, 16, 13, 71, 80, 15, 30, 76, 35, 48, 41, 67])
labels tensor([33, 51, 21, 52, 66, 28, 49, 39, 28, 53, 21, 56, 14, 54, 67, 16])
labels tensor([75, 57,  3, 38, 15, 78, 71, 32, 19, 24,  6, 74, 47,  7, 20, 20])
labels tensor([23, 75, 44, 53,  7, 71, 80, 40, 69, 28, 27, 37, 12, 36, 68, 22])
labels tensor([69, 11, 24, 68, 48, 41,  3, 70, 80, 47, 45, 51, 26, 72, 22, 31])
labels tensor([32, 33, 29, 61, 20, 16, 54, 70, 77, 26, 54,  7, 59,  1, 32, 40])
labels tensor([74, 16, 17, 69, 53,  2,  6, 75, 44,  3, 78, 34, 59, 60,  2, 19])
labels tensor([74, 66, 71,  2,  5, 51, 23, 55, 21, 28, 65, 34, 16, 73,  0, 20])
labels tensor([49, 45, 18, 59, 59, 65, 3

labels tensor([35,  1, 14, 69, 16,  6, 73, 41, 52, 50, 69, 33, 33, 63,  5, 13])
labels tensor([55, 42, 39, 37, 20,  9, 59, 47, 25, 73, 39, 19, 59, 36,  1, 53])
labels tensor([53, 52, 71, 24, 74, 55, 14,  1, 65, 66,  6, 11, 77, 38, 61, 37])
labels tensor([65, 65, 40, 22, 25, 26, 54, 20,  2,  8, 69, 46,  8, 73,  2, 63])
labels tensor([56, 32, 41, 31, 37, 43, 78, 10, 53, 41, 17, 20, 12, 77, 14, 75])
labels tensor([79, 58, 25, 76, 11, 69, 42, 24, 29, 67, 48, 62, 22, 28, 35, 68])
labels tensor([22, 62, 70, 30, 59, 70, 67, 61, 33, 52, 42, 43, 64,  6,  3, 26])
labels tensor([35, 48, 70, 26, 69, 68, 26,  3, 76, 72, 11, 20, 29, 57, 61,  9])
labels tensor([55, 10, 68, 45, 12,  7, 25, 37, 49,  1, 51, 62, 31, 46, 32,  5])
labels tensor([70, 15, 27, 75, 50, 22,  9, 56, 31, 46, 73, 16, 49, 61, 45, 66])
labels tensor([63, 59, 66, 74,  9, 54, 19, 46, 79, 42, 54, 80,  5, 24, 30, 18])
labels tensor([39, 55, 14, 64, 40, 45, 71, 30,  7, 33, 41, 53, 36, 42, 49, 28])
labels tensor([11, 40, 67, 67, 14, 34, 6

labels tensor([21, 75, 28, 38, 75, 56, 80, 78, 36, 61,  1, 58, 51,  7,  9, 51])
labels tensor([33, 67,  1, 39,  2, 43, 68, 29, 25, 16, 11, 69, 47, 21, 31,  4])
labels tensor([14, 43, 69, 59, 67, 73, 62, 54, 11, 58, 31, 10, 20, 31, 27, 58])
labels tensor([ 9, 15, 44, 35, 21, 74, 43, 69, 50, 19, 14, 65, 61, 11, 74, 64])
labels tensor([17,  2,  9,  4, 75,  3,  5,  4, 45, 49, 51, 37, 71, 12, 24, 54])
labels tensor([57, 66, 35, 43, 40, 22, 20, 24, 24, 79, 17,  5, 12,  7, 34, 42])
labels tensor([30,  4, 43, 19,  5, 33, 28, 29, 30, 72, 41, 31, 31, 77,  4, 18])
labels tensor([76, 49, 58, 44, 69,  7, 60, 20, 75, 50, 43, 29, 27, 12, 35, 50])
labels tensor([26,  9, 10, 40, 38, 70, 53,  6, 80, 55, 24, 78, 31,  0, 40, 23])
labels tensor([46, 51, 77, 43,  2, 52, 39, 52,  5, 38, 31, 74, 16, 36, 22, 77])
labels tensor([49, 37, 38, 53, 28, 66, 33, 26, 46,  2,  0, 18, 68, 43, 27,  8])
labels tensor([47, 71, 34, 47, 28, 73, 15, 71, 37,  3, 37, 13, 42, 17, 73, 68])
labels tensor([42, 44, 22, 54, 12, 54, 5

...

labels tensor([46,  7, 32, 38, 62, 75, 46, 58, 61, 31, 24, 59, 74, 23, 30, 69])
labels tensor([71, 11, 36, 52, 47, 41, 77, 37, 41, 70, 46, 11, 27, 59, 51,  3])
labels tensor([42, 65, 12, 20, 45, 72,  0,  1, 66, 41, 49, 80, 35, 80, 24, 55])
labels tensor([51, 62, 10, 33, 25, 66, 75,  2,  4, 75, 40, 43, 21, 22, 53, 31])
labels tensor([65, 32, 63, 60, 59, 62, 26,  9, 16, 27, 68, 39, 13, 47,  0, 45])
labels tensor([78, 33, 36, 55, 18, 65,  2, 11, 20, 50, 79, 35, 20, 24, 41, 69])
labels tensor([47, 26, 53, 34, 28, 37, 73, 17, 52, 64, 61, 14, 47,  9,  5, 65])
labels tensor([ 7, 34, 74,  9, 56, 10, 51,  7, 29, 17, 80, 44, 28, 16,  5, 57])
labels tensor([79, 34, 46, 13, 48, 31, 17, 40, 70, 45, 52, 55, 34, 38, 71, 38])
labels tensor([60, 38, 52, 58,  1,  6, 25, 42, 24, 59, 21, 71, 42, 37, 49, 80])
labels tensor([71, 55, 11,  7, 18, 50, 66, 35, 47, 20, 68, 56, 65, 16,  7,  1])
labels tensor([18, 76, 58, 77, 66, 23, 61, 78, 38, 50, 63, 26, 51, 79,  2, 73])
labels tensor([64, 46,  5,  8, 78, 14, 5

labels tensor([ 0, 40, 60, 77, 44, 39, 25, 71, 73, 28, 42, 27, 17, 27,  5, 61])
labels tensor([47,  2, 24, 14, 32, 11, 59, 48, 71, 33, 49, 27, 72,  0, 79, 55])
labels tensor([67, 78,  3,  9, 26, 43, 52, 42,  2, 59, 17, 51, 21, 57, 78, 32])
labels tensor([13, 66, 77, 12, 69, 20, 43, 61, 70, 24, 60, 80, 74, 33, 42, 78])
labels tensor([80, 29, 67, 25, 59, 49,  9, 51, 17, 16, 58, 80, 55, 73, 42, 35])
labels tensor([28, 36, 28,  9, 15, 76, 49, 15, 33, 48, 78, 62, 57, 75, 35, 75])
labels tensor([31, 58, 20, 73,  9, 66, 33, 16,  7, 62, 17, 19, 12, 53, 58, 70])
labels tensor([65, 61, 44, 21, 75, 50, 44,  3, 79,  1, 20, 72,  8, 20, 54, 71])
labels tensor([11, 10, 35, 56, 56, 10, 20, 27, 55, 55, 67, 37, 45, 73,  4,  7])
labels tensor([53,  4, 80, 18, 17, 37, 45, 73, 68, 65, 79,  2, 76, 16, 76, 30])
labels tensor([49,  4, 33, 16,  6, 67, 76, 59, 69, 51, 76,  6, 59,  6, 66,  8])
labels tensor([45, 65,  4, 46, 77, 38, 62, 37, 63, 61, 32, 57, 67, 27, 54, 65])
labels tensor([ 9,  8, 59, 29, 19,  3, 7

labels tensor([38, 10, 65, 41,  5, 62, 19, 14,  6, 33, 11, 54, 52, 37, 38,  0])
labels tensor([45, 50, 41,  9,  0, 67, 23, 79, 44, 11, 40, 51, 76, 68, 47, 27])
labels tensor([77,  5, 16,  8, 11, 39, 13, 56,  4, 66, 73, 78, 61, 15, 51, 63])
labels tensor([10,  5, 51, 36, 41, 13, 69, 50, 14, 34, 63, 54, 25, 72, 15, 49])
labels tensor([17, 15, 61,  9, 31, 60, 63, 30, 11, 28, 39, 36, 35, 35,  9,  7])
labels tensor([60, 73, 69, 12,  6, 16, 35,  3, 17, 37, 13, 19, 77, 44, 15, 75])
labels tensor([13, 67, 11, 34, 26, 42, 24, 65, 53, 16,  0, 79, 20, 61, 17,  0])
labels tensor([70, 75, 69, 46, 68, 20, 65, 30, 61, 47, 29, 65, 22, 25, 53, 33])
labels tensor([70,  6, 32, 35, 49, 67, 59, 29, 16, 15,  1, 67,  8,  7, 61, 59])
labels tensor([77, 14, 60,  7, 40,  6, 57, 46, 52, 26, 14, 55, 27, 24, 41, 23])
labels tensor([35, 46, 72, 14, 16, 21, 42, 80, 17, 55,  6, 51, 55, 59, 31, 65])
labels tensor([37, 64, 39, 50, 77, 67, 28, 53, 68, 71,  5, 24, 42, 50, 11, 73])
labels tensor([ 0, 79, 76, 39, 62,  5, 6

labels tensor([73, 34, 21,  5, 50, 42, 59, 30,  1, 58, 17, 77, 34, 22, 13, 72])
labels tensor([58, 15,  1,  0, 45, 41, 51, 38, 18,  2, 26, 43, 68, 14,  5, 17])
labels tensor([59, 17, 52, 72, 70, 50, 37, 43,  6, 29, 33, 45, 19, 59, 31, 75])
labels tensor([66, 20, 59, 27, 39, 59,  6, 66, 37,  6, 46, 71,  0, 35, 31, 38])
labels tensor([ 8, 15, 33, 73, 45, 75, 52, 42, 47, 12, 10, 33, 43, 33, 35, 38])
labels tensor([13, 46, 68,  3, 36,  7, 20, 78,  3, 75, 67,  6, 56, 65, 72,  6])
labels tensor([13, 49, 21, 57,  7, 66, 10, 37, 29, 80, 22, 55, 77, 20, 60, 71])
labels tensor([ 1,  2, 11, 39, 64, 19,  1,  4, 67, 33, 76, 13,  9, 30, 35, 48])
labels tensor([ 8,  4, 20, 70, 21, 42, 76, 43, 16, 43, 50, 29, 60, 64, 35, 69])
labels tensor([44, 66, 73, 66, 10, 26, 79, 24, 36, 57, 74,  0, 13, 12, 41, 38])
labels tensor([38, 19, 61,  8, 25, 52, 10, 62, 51, 63, 78, 17, 60,  2, 46, 22])
labels tensor([53, 50,  9,  5, 48, 19, 42, 18,  4, 77, 23, 14, 64, 50, 70, 22])
labels tensor([45, 28, 38, 11,  3, 46, 3

labels tensor([30, 58, 28, 77,  6, 32, 49, 26, 39, 28, 45, 67,  1, 61,  3, 26])
labels tensor([19, 52, 80, 32, 32, 42, 14, 80, 39, 14, 59, 80, 49, 35, 21, 52])
labels tensor([35, 34,  0, 76, 35, 13, 78, 70, 57, 64, 78,  4, 38, 50, 44, 69])
labels tensor([13, 77, 66,  7, 15, 60, 56, 56, 39, 61, 58, 22, 26, 63, 22, 10])
labels tensor([35, 62, 35,  7, 79, 66,  4, 40,  2, 45, 12, 41, 19, 21, 41, 19])
labels tensor([28, 73, 68, 24, 32,  9, 28, 36, 59, 31,  3, 45,  7, 43, 31, 32])
labels tensor([55, 73, 77, 12, 29,  1, 50, 22, 13, 24, 58, 21, 18, 12, 64,  9])
labels tensor([19, 67, 12, 39, 34, 17,  8, 65, 70,  4, 22, 20, 25, 71, 63, 17])
labels tensor([52, 71, 31, 71, 73, 80, 39, 31,  3, 45, 38,  0, 68, 50, 38, 25])
labels tensor([49, 42, 43, 23, 11, 71,  2, 31, 20,  6, 28, 38,  3, 55, 70,  5])
labels tensor([42, 39,  8, 79, 52,  1, 24,  8, 54, 29, 62, 28,  9, 17, 43, 73])
labels tensor([75, 75, 42, 20, 53, 59,  1, 29, 60, 22,  0, 51, 20, 73, 21, 11])
labels tensor([68, 53,  9, 49, 48,  9,  

In [22]:
for batch_data, batch_labels in dataloader:
    print("Batch Data:", batch_data)
    print("Batch Labels:", batch_labels)

Batch Data: tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]],


        [[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1

Batch Data: tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]],


        [[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1

Batch Data: tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]],


        [[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1

Batch Data: tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]],


        [[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1

Batch Data: tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]],


        [[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1

...

Batch Data: tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]],


        [[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1

Batch Data: tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]],


        [[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1

Batch Data: tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]],


        [[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1

Batch Data: tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]],


        [[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1

Batch Data: tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],

         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]],


        [[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1

In [55]:
# ...Testing

# Define the testing dataset and DataLoader (similar to training)
test_data_folder = r"..\CreatedDatasets\logMelspectrogram_TestSad" # Update with
#the path to your test spectrogram images

transform = transforms.Compose([
    transforms.Resize((490, 990)),  # Resize images to new_height x new_width
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize images
])
test_dataset = ImageFolder(test_data_folder, transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False) # Set shuffle to
print("batchsize: ",batch_size)
print(test_dataset)
#False for the test set

# Testing loop
model.eval() # Set the model to evaluation mode (no gradient calculation)
total_test_correct = 0
total_test_samples = 0

with torch.no_grad(): # No gradient calculation during testing
    for data, labels in test_dataloader:
        print("Actual Labels:", labels)
        data=data.to(device, dtype=torch.float)
        labels=labels.to(device)
        test_outputs = model(data)
        _, test_predicted = torch.max(test_outputs, 1)
        total_test_correct += (test_predicted == labels).sum().item()
        
        total_test_samples += labels.size(0)

test_accuracy = total_test_correct / total_test_samples
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Set the model back to training mode
model.train()

print("Training and Testing complete")

batchsize:  16
Dataset ImageFolder
    Number of datapoints: 161
    Root location: ..\CreatedDatasets\logMelspectrogram_TestSad
    StandardTransform
Transform: Compose(
               Resize(size=(490, 990), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )
Actual Labels: tensor([0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 8])
Actual Labels: tensor([ 8,  9,  9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16])
Actual Labels: tensor([16, 17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23, 24])
Actual Labels: tensor([24, 25, 25, 26, 26, 27, 27, 28, 28, 29, 29, 30, 30, 31, 31, 32])
Actual Labels: tensor([32, 33, 33, 34, 34, 35, 35, 36, 36, 37, 37, 38, 38, 39, 39, 40])
Actual Labels: tensor([40, 41, 41, 42, 42, 43, 43, 44, 44, 45, 45, 46, 46, 47, 47, 48])
Actual Labels: tensor([48, 49, 49, 50, 50, 51, 51, 52, 52, 53, 53, 54, 54, 55, 55, 56])
Actual Labels: tensor([56, 5

In [30]:
torch.save(model.state_dict(), '../cremaD_saved_Models/logmel-Resnet.pth')

In [56]:
crema_D_NN=100  #samedata=100%
crema_D_NN=93.17 
crema_D_NA=22.36
crema_D_NF=40.37
crema_D_NS=72.67
crema_D_NH=42.86
